# InfoGAN CeleA

[InfoGAN: Interpretable Representation Learning by Information Maximizing Generative Adversarial Nets](https://arxiv.org/abs/1606.03657).


In [1]:
#Import the libraries we will need.
import os, glob, cv2, math, sys
import tensorflow as tf
import numpy as np
from numpy.random import permutation
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import scipy.misc
import scipy
#from PIL import Image

np.random.seed(1)
#plt.ion()   # interactive mode
%matplotlib inline

load CeleA data

In [2]:
# http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_image_display/py_image_display.html
#img_rows, img_cols = 100, 100
#img_rows, img_cols = 64, 64
img_rows, img_cols = 32, 32

def get_im(path):

    #img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = cv2.imread(path)
    #img = plt.imread(path)
    #resized = img
    resized = cv2.resize(img, (img_cols, img_rows))

    return resized

def read_train_data_fullname(path):

    
    files = glob.glob(path)
    # shuffling
    filenames = permutation(files)
    
    return filenames

def read_train_data_fullname_lfw(path):

    root = path
    all_folders = os.listdir(root)
    path = []
    for afolder in all_folders:
        path.append(root+"/"+afolder+"/*.jpg")
    #print(path)
    files = []
    for apath in path:
        templist = glob.glob(apath)
        for afile in templist:
            files.append(afile)

    # shuffling
    filenames = permutation(files)
    
    return filenames

def read_train_data_batch(filenames, batchsize=5):
    
    end = min(len(filenames), batchsize)
    train_data = []
    
    for fl in filenames[:end]:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        img = np.array(img, dtype=np.float32)
        
        #normalization
        #img -= np.mean(img)
        #img /= np.std(img)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)

    # dataid, height, width, #channel -> dataid, #channel, height, width
    #train_data = train_data.transpose((0, 3, 1, 2))
    filenames = filenames[end:]
    
    return train_data, filenames

def read_train_data_mini_batch(filenames, startpoint, batchsize=5):
    
    train_data = []
    
    for fl in filenames[startpoint:startpoint+batchsize]:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        img = np.array(img, dtype=np.float32)
        #print(type(img))        
        #normalization
        img -= np.mean(img)
        img /= np.std(img)
        #print(img.shape)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)
    
    return train_data

#random mini-batch
def read_train_data_random_batch(filenames, batchsize=5):
    fullsize=len(filenames)
    #http://qiita.com/hamukazu/items/ec1b4659df00f0ce43b1
    idset = np.random.randint(0, high=fullsize, size=batchsize)
    #print(idset) 
    train_data = []
    
    for fid in idset:
        fl = filenames[fid]
        flbase = os.path.basename(fl)
        img = get_im(fl)
        #img[:,:,1] = 0
        #img[:,:,2] = 0
        img = np.array(img, dtype=np.float32)
        #img = np.reshape(img,[img.shape[0],img.shape[1],1])
        # normalization
        # https://stats.stackexchange.com/questions/185853/why-do-we-need-to-normalize-the-images-before-we-put-them-into-cnn
        img -= np.mean(img)
        img /= np.std(img)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)

    # dataid, height, width, #channel -> dataid, #channel, height, width
    #train_data = train_data.transpose((0, 3, 1, 2))
    
    return train_data
# must be full path, ~/... not ok
# train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba/*.jpg")
#train_data,train_data_filenames = read_train_data_batch(train_data_filenames)

#random mini-batch
#train_data = read_train_data_random_batch(train_data_filenames)

In [3]:
#print(len(train_data),len(train_data_filenames))
#print(len(train_data[0][1]))

def imshow(inp, title=None):
    """Imshow for Tensor."""
    # need this transpose if there is transpose in read_train_data_xx calls
    #inp = inp.numpy().transpose((1, 2, 0))
    #inp = std * inp + mean
    #plt.imshow(inp,cmap="Purples",interpolation = 'bicubic')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.01)  # pause a bit so that plots are updated

#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba/*.jpg")
#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba_png.7z/img_align_celeba_png/*.png")
#train_data_filenames = read_train_data_fullname_lfw("/home/cli/LFW/lfw2")
#print(len(train_data_filenames))
#print(train_data_filenames[3])
#train_data = read_train_data_mini_batch(train_data_filenames,0)  
#print(train_data[0].shape)
#print(train_data[43][:,:,0:2].shape)

#imshow(train_data[2][:,:,0])
#imshow(train_data[2][:,:,0])

### Helper Functions

In [4]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def lrelu(x, leak=0.2, name="lrelu"):
     with tf.variable_scope(name):
         f1 = 0.5 * (1 + leak)
         f2 = 0.5 * (1 - leak)
         return f1 * x + f2 * abs(x)
    
#The below functions are taken from carpdem20's implementation https://github.com/carpedm20/DCGAN-tensorflow
#They allow for saving sample images from the generator to follow progress
def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge_color(images, size))
    #return scipy.misc.imsave(path, merge(images, size))

def inverse_transform(images):
    #return images
    # https://zhuanlan.zhihu.com/p/25051313
    return (images+1.)/2.

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image

    return img

def merge_color(images, size):
    h, w, c = images.shape[1], images.shape[2],images.shape[3]
    img = np.zeros((h * size[0], w * size[1],c))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image[:,:,:]

    return img


## Defining the Adversarial Networks

### Generator Network

The generator takes a vector of random numbers and transforms it into a 32x32 image. Each layer in the network involves a strided  transpose convolution, batch normalization, and rectified nonlinearity. Tensorflow's slim library allows us to easily define each of these layers.

In [5]:
# https://www.tensorflow.org/api_docs/python/tf/depth_to_space
# http://qiita.com/tadOne/items/48302a399dcad44c69c8   Tensorflow - padding = VALID/SAMEの違いについて
#     so 3 tf.depth_to_space(genX,2) gives 4x2^3 = 32
# 

def generator(z):
    
    zP = slim.fully_connected(z,4*4*448,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,4,4,448])
    
    gen1 = slim.convolution2d(\
        zCon,num_outputs=256,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv1', weights_initializer=initializer)
    gen1 = tf.depth_to_space(gen1,2)
    
    gen2 = slim.convolution2d(\
        gen1,num_outputs=128,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv2', weights_initializer=initializer)
    gen2 = tf.depth_to_space(gen2,2)
    
    gen3 = slim.convolution2d(\
        gen2,num_outputs=64,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv3', weights_initializer=initializer)
    gen3 = tf.depth_to_space(gen3,2)
    
    g_out = slim.convolution2d(\
        gen3,num_outputs=3,kernel_size=[32,32],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_out', weights_initializer=initializer)
    
    return g_out

### Discriminator Network
The discriminator network takes as input a 32x32 image and transforms it into a single valued probability of being generated from real-world data. Again we use tf.slim to define the convolutional layers, batch normalization, and weight initialization.

In [6]:
def discriminator(bottom, cat_list,conts, reuse=False):
    
    dis1 = slim.convolution2d(bottom,64,[4,4],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    dis1 = tf.space_to_depth(dis1,2)
    
    dis2 = slim.convolution2d(dis1,128,[4,4],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    dis2 = tf.space_to_depth(dis2,2)
    
    dis3 = slim.convolution2d(dis2,256,[4,4],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    dis3 = tf.space_to_depth(dis3,2)
        
    dis4 = slim.fully_connected(slim.flatten(dis3),1024,activation_fn=lrelu,\
        reuse=reuse,scope='d_fc1', weights_initializer=initializer)
        
    d_out = slim.fully_connected(dis4,1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    q_a = slim.fully_connected(dis4,128,normalizer_fn=slim.batch_norm,\
        reuse=reuse,scope='q_fc1', weights_initializer=initializer)
    
    
    ## Here we define the unique layers used for the q-network. The number of outputs depends on the number of 
    ## latent variables we choose to define.
    q_cat_outs = []
    for idx,var in enumerate(cat_list):
        q_outA = slim.fully_connected(q_a,var,activation_fn=tf.nn.softmax,\
            reuse=reuse,scope='q_out_cat_'+str(idx), weights_initializer=initializer)
        q_cat_outs.append(q_outA)
    
    q_cont_outs = None
    if conts > 0:
        q_cont_outs = slim.fully_connected(q_a,conts,activation_fn=tf.nn.tanh,\
            reuse=reuse,scope='q_out_cont_'+str(conts), weights_initializer=initializer)
    
    #print("d_out"+str(d_out))
    return d_out,q_cat_outs,q_cont_outs

### Connecting them together

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/split
# https://www.tensorflow.org/api_docs/python/tf/one_hot
# https://www.tensorflow.org/api_docs/python/tf/concat
# https://www.tensorflow.org/api_docs/python/tf/reduce_sum
# https://www.tensorflow.org/api_docs/python/tf/reduce_mean
# https://www.tensorflow.org/api_docs/python/tf/trainable_variables
# https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer
# https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm
# https://deepage.net/deep_learning/2016/10/26/batch_normalization.html
# z_lat: one_hot_size + z_size + number_continuous = 10+64+2=76
# g_loss def is interesting, my understanding: 
#        if Dg is the probablity to be told as feak data, then 1-Dg is the probabily of suceessfully cheating, 
#        so we cal KL(Dg/(1-Dg)), and readuce_mean works as sampling proceduce
# 

tf.reset_default_graph()


z_size = 128 #Size of initial z vector used for generator.

# Define latent variables.
#categorical_list = [10]*10 # Each entry in this list defines a categorical variable of a specific size.
categorical_list = [10] # Each entry in this list defines a categorical variable of a specific size.
# categorical_list = [10,10] # Each entry in this list defines a categorical variable of a specific size.
number_continuous = 1 # The number of continous variables.

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)

#These placeholders are used for input into the generator and discriminator, respectively.
z_in = tf.placeholder(shape=[None,z_size],dtype=tf.float32) #Random vector
real_in = tf.placeholder(shape=[None,32,32,3],dtype=tf.float32) #Real images

#These placeholders load the latent variables.
latent_cat_in = tf.placeholder(shape=[None,len(categorical_list)],dtype=tf.int32)
#print("latent_cat_in:", latent_cat_in)
latent_cat_list = tf.split(latent_cat_in,len(categorical_list),1)
#print("latent_cat_list: ",latent_cat_list)
if number_continuous>0:
    latent_cont_in = tf.placeholder(shape=[None,number_continuous],dtype=tf.float32)

oh_list = []
for idx,var in enumerate(categorical_list):
    latent_oh = tf.one_hot(tf.reshape(latent_cat_list[idx],[-1]),var)
    #print(latent_cat_list[idx])
    #print(latent_oh),  woundn't print anything in sess.run()
    oh_list.append(latent_oh)

#Concatenate all c and z variables.
z_lats = oh_list[:]
#print("1st z_lats: ", z_lats )
z_lats.append(z_in)
#print("2nd z_lats: ", z_lats )
if number_continuous>0:
    z_lats.append(latent_cont_in)
#print("3rd z_lats: ", z_lats )
z_lat = tf.concat(z_lats,1)
#print("z_lat: ", z_lat )

Gz = generator(z_lat) #Generates images from random z vectors
#print (Gz.shape)
Dx,_,_ = discriminator(real_in,categorical_list,number_continuous) #Produces probabilities for real images
Dg,QgCat,QgCont = discriminator(Gz,categorical_list,number_continuous,reuse=True) #Produces probabilities for generator images

#These functions together define the optimization objective of the GAN.
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg)) #This optimizes the discriminator.
#g_loss = -tf.reduce_mean(tf.log((Dg/(1.-Dg)))) #KL Divergence optimizer
g_loss = -tf.reduce_mean(tf.log(Dg)) 

#Combine losses for each of the categorical variables.
cat_losses = []
for idx,latent_var in enumerate(oh_list):
    #print ("latent_var: ", latent_var)
    #print ("tf.log(QgCat[idx]): ",tf.log(QgCat[idx]))
    cat_loss = -tf.reduce_sum(latent_var*tf.log(QgCat[idx]),axis=1)
    cat_losses.append(cat_loss)
    
#Combine losses for each of the continous variables.
if number_continuous > 0:
    q_cont_loss = tf.reduce_sum(0.5 * tf.square(latent_cont_in - QgCont),axis=1)
else:
    q_cont_loss = tf.constant(0.0)

q_cont_loss = tf.reduce_mean(q_cont_loss)
q_cat_loss = tf.reduce_mean(cat_losses)
q_loss = tf.add(q_cat_loss,q_cont_loss)
tvars = tf.trainable_variables()
#print (len(tvars))
#for i in tvars:
#    print(i)

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.4)
trainerQ = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
#
d_grads = trainerD.compute_gradients(d_loss,tvars[9:-2-((number_continuous>0)*2)-(len(categorical_list)*2)]) #Only update the weights for the discriminator network.
g_grads = trainerG.compute_gradients(g_loss, tvars[0:9]) #Only update the weights for the generator network.
q_grads = trainerQ.compute_gradients(q_loss, tvars) 

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)
update_Q = trainerQ.apply_gradients(q_grads)

## Training the network
Now that we have fully defined our network, it is time to train it!

In [ ]:
# on at52 (GTX1080), 15mins/10000 epochs , 5000000 is about 12.5 hrs　 
# https://stackoverflow.com/questions/19349410/how-to-pad-with-zeros-a-tensor-along-some-axis-python
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html
# blow up after 81800
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html
# https://www.tensorflow.org/api_docs/python/tf/Session#run
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.hstack.html
c_val = 10

batch_size = 64 #Size of image batch to apply at each iteration.
#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba_png.7z/img_align_celeba_png/*.png")
train_data_filenames = read_train_data_fullname_lfw("/home/cli/LFW/lfw2")

#iterations = 500000 #Total number of iterations to use.
iterations = 1000 #Total number of iterations to use.
sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to save trained model to.

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(init)
    for i in range(iterations):
        train_data_filenames=permutation(train_data_filenames) # mini-batch
        data_left = len(train_data_filenames)
        batch_counter = 0
        while data_left>0:
            batch_size_to_train = min(batch_size, data_left)          
            
            zs = np.random.uniform(-1.0,1.0,size=[batch_size_to_train,z_size]).astype(np.float32) #Generate a random z batch
            #print("zs shape:",zs.shape)
            
            #lcat = np.random.randint(0,10,[batch_size,len(categorical_list)]) #Generate random c batch
            lcat = np.random.randint(0,c_val,[batch_size_to_train,len(categorical_list)]) #Generate random c batch
            
            lcont = np.random.uniform(-1,1,[batch_size_to_train,number_continuous]) #

            #xs = read_train_data_random_batch(train_data_filenames, batchsize=batch_size_to_train)
            xs = read_train_data_mini_batch(train_data_filenames, batch_counter*batch_size, batch_size_to_train)
            
            _,dLoss = sess.run([update_D,d_loss],feed_dict={z_in:zs,real_in:xs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the discriminator
            _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the generator, twice for good measure.
            _,qLoss,qK,qC = sess.run([update_Q,q_loss,q_cont_loss,q_cat_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update to optimize mutual information.

            data_left = data_left - batch_size_to_train
            batch_counter +=1
            if batch_counter%1 == 0 or data_left == 0:
                z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32)
                lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
                a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
                b = np.reshape(a,[c_val*c_val,1])
                lcont_sample = b
                samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample})
                if not os.path.exists(sample_directory):
                    os.makedirs(sample_directory)
                save_images(np.reshape(samples[0:100],[100,32,32,3]),[10,10],sample_directory+'/fig'\
                            +str(i)+'_'+str(batch_counter)+'.png')
                
                print ("epoch:"+str(i)+" batch_done:"+str(batch_counter) \
                       +" Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss) + " Q Losses: " + str([qK,qC]))
            
             
        """
        if i % 100 == 0:
            print ("Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss) + " Q Losses: " + str([qK,qC]))
            #z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
            z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32) #Generate another z batch
            #lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
            lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
            latent_fixed = np.ones((c_val*c_val,1))
            lcat_sample = np.hstack([latent_fixed,lcat_sample])
            
            #a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
            a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
            #b = np.reshape(a,[100,1])
            b = np.reshape(a,[c_val*c_val,1])
            c = np.zeros_like(b)
            lcont_sample = np.hstack([b,c])
            #
            samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
            if not os.path.exists(sample_directory):
                os.makedirs(sample_directory)
            #Save sample generator images for viewing training progress.
            #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig'+str(i)+'.png')
            save_images(np.reshape(samples[0:c_val*c_val],[c_val*c_val,32,32]),[c_val,c_val],sample_directory+'/fig'+str(i)+'.png')
        """
        
        if i % 10 == 0 and i != 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')
            print ("Saved Model on ", i)


epoch:0 batch_done:1 Gen Loss: 3.97692 Disc Loss: 1.65535 Q Losses: [0.2209909, 2.3021722]
epoch:0 batch_done:2 Gen Loss: 10.6725 Disc Loss: 0.983071 Q Losses: [0.15692919, 2.2700653]
epoch:0 batch_done:3 Gen Loss: 7.2922 Disc Loss: 0.411932 Q Losses: [0.13837145, 2.3361697]
epoch:0 batch_done:4 Gen Loss: 12.2727 Disc Loss: 1.28163 Q Losses: [0.13925597, 2.2842112]
epoch:0 batch_done:5 Gen Loss: 5.10141 Disc Loss: 2.20942 Q Losses: [0.12259919, 2.2668846]
epoch:0 batch_done:6 Gen Loss: 11.0006 Disc Loss: 1.5264 Q Losses: [0.11910344, 2.2542524]
epoch:0 batch_done:7 Gen Loss: 11.0247 Disc Loss: 0.289347 Q Losses: [0.11063288, 2.2532749]
epoch:0 batch_done:8 Gen Loss: 5.92379 Disc Loss: 0.151191 Q Losses: [0.10034138, 2.2489665]
epoch:0 batch_done:9 Gen Loss: 14.6506 Disc Loss: 2.32099 Q Losses: [0.095879316, 2.2115786]
epoch:0 batch_done:10 Gen Loss: 10.8081 Disc Loss: 2.18877 Q Losses: [0.11042821, 2.2312627]
epoch:0 batch_done:11 Gen Loss: 2.65694 Disc Loss: 0.640113 Q Losses: [0.1139

epoch:0 batch_done:90 Gen Loss: 33.3241 Disc Loss: 4.86764 Q Losses: [0.041059278, 0.90894228]
epoch:0 batch_done:91 Gen Loss: 14.155 Disc Loss: 2.84692 Q Losses: [0.063347615, 0.89060712]
epoch:0 batch_done:92 Gen Loss: 2.53305 Disc Loss: 0.197935 Q Losses: [0.056810722, 0.89171875]
epoch:0 batch_done:93 Gen Loss: 32.5968 Disc Loss: 1.9267 Q Losses: [0.061246894, 0.81116462]
epoch:0 batch_done:94 Gen Loss: 33.9135 Disc Loss: 0.777237 Q Losses: [0.057007972, 0.83628476]
epoch:0 batch_done:95 Gen Loss: 16.6724 Disc Loss: 0.878904 Q Losses: [0.041109014, 0.89850664]
epoch:0 batch_done:96 Gen Loss: 2.3732 Disc Loss: 0.468039 Q Losses: [0.07167162, 0.90092003]
epoch:0 batch_done:97 Gen Loss: 30.7628 Disc Loss: 1.88397 Q Losses: [0.050247967, 0.82712775]
epoch:0 batch_done:98 Gen Loss: 24.5014 Disc Loss: 1.22442 Q Losses: [0.065144889, 0.86233675]
epoch:0 batch_done:99 Gen Loss: 9.22656 Disc Loss: 1.01283 Q Losses: [0.052137263, 0.83936095]
epoch:0 batch_done:100 Gen Loss: 2.0657 Disc Loss:

epoch:0 batch_done:176 Gen Loss: 5.37506 Disc Loss: 0.00347742 Q Losses: [0.04479963, 0.36037925]
epoch:0 batch_done:177 Gen Loss: 26.6788 Disc Loss: 0.597342 Q Losses: [0.029297462, 0.37689903]
epoch:0 batch_done:178 Gen Loss: 27.7698 Disc Loss: 0.286969 Q Losses: [0.047075674, 0.40553933]
epoch:0 batch_done:179 Gen Loss: 17.2314 Disc Loss: 0.801434 Q Losses: [0.04028663, 0.4209488]
epoch:0 batch_done:180 Gen Loss: 8.7777 Disc Loss: 0.0264434 Q Losses: [0.033635534, 0.47222084]
epoch:0 batch_done:181 Gen Loss: 5.46594 Disc Loss: 0.0279857 Q Losses: [0.050270788, 0.45643392]
epoch:0 batch_done:182 Gen Loss: 4.49545 Disc Loss: 0.0284025 Q Losses: [0.0397055, 0.39179748]
epoch:0 batch_done:183 Gen Loss: 5.0188 Disc Loss: 0.0438434 Q Losses: [0.029009255, 0.35765955]
epoch:0 batch_done:184 Gen Loss: 18.9641 Disc Loss: 0.356099 Q Losses: [0.037482325, 0.38721508]
epoch:0 batch_done:185 Gen Loss: 16.0383 Disc Loss: 0.860929 Q Losses: [0.034430098, 0.36784133]
epoch:0 batch_done:186 Gen Loss

epoch:1 batch_done:55 Gen Loss: 16.1963 Disc Loss: 0.357467 Q Losses: [0.015795611, 0.21564101]
epoch:1 batch_done:56 Gen Loss: 10.4581 Disc Loss: 0.0807641 Q Losses: [0.017458092, 0.19386521]
epoch:1 batch_done:57 Gen Loss: 4.82153 Disc Loss: 0.0356646 Q Losses: [0.026029818, 0.19694282]
epoch:1 batch_done:58 Gen Loss: 34.132 Disc Loss: 1.06864 Q Losses: [0.021729067, 0.19329049]
epoch:1 batch_done:59 Gen Loss: 37.1161 Disc Loss: 1.3671 Q Losses: [0.022324938, 0.21441987]
epoch:1 batch_done:60 Gen Loss: 25.2984 Disc Loss: 1.43301 Q Losses: [0.020200614, 0.20780179]
epoch:1 batch_done:61 Gen Loss: 9.8901 Disc Loss: 0.0102783 Q Losses: [0.021233507, 0.19355419]
epoch:1 batch_done:62 Gen Loss: 2.62085 Disc Loss: 0.0179449 Q Losses: [0.029470418, 0.20428476]
epoch:1 batch_done:63 Gen Loss: 26.7449 Disc Loss: 0.513204 Q Losses: [0.029343808, 0.20411414]
epoch:1 batch_done:64 Gen Loss: 26.3312 Disc Loss: 0.347303 Q Losses: [0.022258574, 0.18539158]
epoch:1 batch_done:65 Gen Loss: 19.6178 Di

epoch:1 batch_done:140 Gen Loss: 6.52589 Disc Loss: 0.064535 Q Losses: [0.021165196, 0.13622178]
epoch:1 batch_done:141 Gen Loss: 31.4293 Disc Loss: 0.356229 Q Losses: [0.019496107, 0.13823506]
epoch:1 batch_done:142 Gen Loss: 31.3211 Disc Loss: 0.251205 Q Losses: [0.017552853, 0.12145579]
epoch:1 batch_done:143 Gen Loss: 20.2397 Disc Loss: 0.701937 Q Losses: [0.014877701, 0.12607917]
epoch:1 batch_done:144 Gen Loss: 5.46077 Disc Loss: 0.00900299 Q Losses: [0.014964756, 0.16517666]
epoch:1 batch_done:145 Gen Loss: 43.4247 Disc Loss: 4.55544 Q Losses: [0.023899473, 0.17001411]
epoch:1 batch_done:146 Gen Loss: 28.8302 Disc Loss: 3.27709 Q Losses: [0.018575443, 0.14639716]
epoch:1 batch_done:147 Gen Loss: 12.9159 Disc Loss: 0.754349 Q Losses: [0.027956584, 0.14567956]
epoch:1 batch_done:148 Gen Loss: 2.36265 Disc Loss: 0.340757 Q Losses: [0.018393341, 0.19793651]
epoch:1 batch_done:149 Gen Loss: 13.8243 Disc Loss: 0.348788 Q Losses: [0.026354574, 0.16645341]
epoch:1 batch_done:150 Gen Los

...

epoch:8 batch_done:23 Gen Loss: 6.86714 Disc Loss: 0.00249779 Q Losses: [0.0073753488, 0.0082580503]
epoch:8 batch_done:24 Gen Loss: 7.06267 Disc Loss: 0.00188649 Q Losses: [0.005019566, 0.0064837425]
epoch:8 batch_done:25 Gen Loss: 6.92333 Disc Loss: 0.00165422 Q Losses: [0.0052534826, 0.0064046723]
epoch:8 batch_done:26 Gen Loss: 6.93505 Disc Loss: 0.00173746 Q Losses: [0.0068221167, 0.0067293867]
epoch:8 batch_done:27 Gen Loss: 7.09853 Disc Loss: 0.00142644 Q Losses: [0.0060605332, 0.0072180964]
epoch:8 batch_done:28 Gen Loss: 7.28082 Disc Loss: 0.00135339 Q Losses: [0.0055463724, 0.0060213935]
epoch:8 batch_done:29 Gen Loss: 7.75184 Disc Loss: 0.000625524 Q Losses: [0.0062375031, 0.0075720837]
epoch:8 batch_done:30 Gen Loss: 7.96133 Disc Loss: 0.00049578 Q Losses: [0.0085636023, 0.0091410633]
epoch:8 batch_done:31 Gen Loss: 7.69148 Disc Loss: 0.000739533 Q Losses: [0.0055930587, 0.005411061]
epoch:8 batch_done:32 Gen Loss: 7.56531 Disc Loss: 0.000664991 Q Losses: [0.010311676, 0.00

epoch:8 batch_done:104 Gen Loss: 7.25713 Disc Loss: 0.0010799 Q Losses: [0.0054446827, 0.0067939525]
epoch:8 batch_done:105 Gen Loss: 7.75319 Disc Loss: 0.000622322 Q Losses: [0.0075615854, 0.0085721053]
epoch:8 batch_done:106 Gen Loss: 8.23835 Disc Loss: 0.000340576 Q Losses: [0.0087881153, 0.0069152452]
epoch:8 batch_done:107 Gen Loss: 8.36509 Disc Loss: 0.000296591 Q Losses: [0.0056427261, 0.0066111935]
epoch:8 batch_done:108 Gen Loss: 8.05029 Disc Loss: 0.000451739 Q Losses: [0.0047481824, 0.0074281916]
epoch:8 batch_done:109 Gen Loss: 9.4066 Disc Loss: 0.000254266 Q Losses: [0.0053429105, 0.0076785041]
epoch:8 batch_done:110 Gen Loss: 9.59822 Disc Loss: 0.000311631 Q Losses: [0.0070568477, 0.0081174392]
epoch:8 batch_done:111 Gen Loss: 7.57251 Disc Loss: 0.00069513 Q Losses: [0.0099301431, 0.008305382]
epoch:8 batch_done:112 Gen Loss: 7.5636 Disc Loss: 0.00064924 Q Losses: [0.0078685274, 0.0070020072]
epoch:8 batch_done:113 Gen Loss: 7.38863 Disc Loss: 0.000928954 Q Losses: [0.013

epoch:8 batch_done:185 Gen Loss: 7.6607 Disc Loss: 0.000800186 Q Losses: [0.013981825, 0.011799883]
epoch:8 batch_done:186 Gen Loss: 8.59928 Disc Loss: 0.00103706 Q Losses: [0.0099817198, 0.012473162]
epoch:8 batch_done:187 Gen Loss: 11.4 Disc Loss: 9.02317e-05 Q Losses: [0.0075219106, 0.015124094]
epoch:8 batch_done:188 Gen Loss: 15.5718 Disc Loss: 6.40622e-05 Q Losses: [0.0084066149, 0.018002136]
epoch:8 batch_done:189 Gen Loss: 12.429 Disc Loss: 0.000270943 Q Losses: [0.0074464115, 0.011461616]
epoch:8 batch_done:190 Gen Loss: 9.84611 Disc Loss: 0.000102487 Q Losses: [0.0059577245, 0.0097381203]
epoch:8 batch_done:191 Gen Loss: 9.30604 Disc Loss: 0.000112291 Q Losses: [0.008295048, 0.010370598]
epoch:8 batch_done:192 Gen Loss: 8.85952 Disc Loss: 0.000225288 Q Losses: [0.010607991, 0.006708255]
epoch:8 batch_done:193 Gen Loss: 8.80742 Disc Loss: 0.000197578 Q Losses: [0.0076492294, 0.011880966]
epoch:8 batch_done:194 Gen Loss: 8.89131 Disc Loss: 0.0002077 Q Losses: [0.0078092865, 0.0

epoch:9 batch_done:60 Gen Loss: 8.76384 Disc Loss: 0.000329831 Q Losses: [0.012756507, 0.0050327587]
epoch:9 batch_done:61 Gen Loss: 7.03563 Disc Loss: 0.0014441 Q Losses: [0.0055086724, 0.0084803142]
epoch:9 batch_done:62 Gen Loss: 6.90544 Disc Loss: 0.00240659 Q Losses: [0.0098870378, 0.0067510186]
epoch:9 batch_done:63 Gen Loss: 7.49807 Disc Loss: 0.00136565 Q Losses: [0.0058409162, 0.0068575349]
epoch:9 batch_done:64 Gen Loss: 8.11221 Disc Loss: 0.00266336 Q Losses: [0.011380978, 0.0063553606]
epoch:9 batch_done:65 Gen Loss: 9.41036 Disc Loss: 0.000735784 Q Losses: [0.0060763434, 0.0058759255]
epoch:9 batch_done:66 Gen Loss: 9.81499 Disc Loss: 0.000265409 Q Losses: [0.0057485066, 0.006650717]
epoch:9 batch_done:67 Gen Loss: 7.26172 Disc Loss: 0.00146402 Q Losses: [0.006883217, 0.0068705878]
epoch:9 batch_done:68 Gen Loss: 6.79523 Disc Loss: 0.00267526 Q Losses: [0.0084398985, 0.0077645257]
epoch:9 batch_done:69 Gen Loss: 7.56604 Disc Loss: 0.00106489 Q Losses: [0.0066315606, 0.0082

epoch:9 batch_done:144 Gen Loss: 6.69885 Disc Loss: 0.00423681 Q Losses: [0.012956385, 0.0065321019]
epoch:9 batch_done:145 Gen Loss: 6.58842 Disc Loss: 0.0072379 Q Losses: [0.0036479416, 0.0071142204]
epoch:9 batch_done:146 Gen Loss: 6.09906 Disc Loss: 0.0222944 Q Losses: [0.012888747, 0.009341348]
epoch:9 batch_done:147 Gen Loss: 6.63526 Disc Loss: 0.0111494 Q Losses: [0.0058408715, 0.0059065805]
epoch:9 batch_done:148 Gen Loss: 7.06554 Disc Loss: 0.00511508 Q Losses: [0.005088876, 0.0065969154]
epoch:9 batch_done:149 Gen Loss: 7.22314 Disc Loss: 0.0028828 Q Losses: [0.0099478662, 0.0085976701]
epoch:9 batch_done:150 Gen Loss: 7.03544 Disc Loss: 0.00246766 Q Losses: [0.0091863722, 0.0095257852]
epoch:9 batch_done:151 Gen Loss: 6.91186 Disc Loss: 0.0024317 Q Losses: [0.0083575975, 0.0071993242]
epoch:9 batch_done:152 Gen Loss: 6.77817 Disc Loss: 0.00281906 Q Losses: [0.0071761594, 0.0059642959]
epoch:9 batch_done:153 Gen Loss: 6.60217 Disc Loss: 0.00408549 Q Losses: [0.013569463, 0.00

## Using a trained network
Once we have a trained model saved, we may want to use it to generate new images, and explore the representation it has learned.

In [ ]:
# http://qiita.com/TokyoMickey/items/f6a9251f5a59120e39f8
"""
sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to load trained model from.

#init = tf.initialize_all_variables()
c_val = 10

saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())
    #sess.run(init)
    #Reload the model.
    print ('Loading Model...')
    ckpt = tf.train.get_checkpoint_state(model_directory)
    saver.restore(sess,ckpt.model_checkpoint_path)
    
    #z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
    z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32) #Generate another z batch
    #lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
    #lcat_sample = np.reshape(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]),[100,1])
    lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
    #print(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]))
    #latent_fixed = np.ones((c_val*c_val,1))*50
    latent_fixed = np.zeros((c_val*c_val,1))
    #lcat_sample = np.hstack([latent_fixed,lcat_sample])
    # good shape
    lcat_sample = np.hstack([lcat_sample,latent_fixed])
            
    #a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
    a = a = np.reshape(np.array([[(e*5/4.5 - 5.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
    #a = a = np.ones((c_val*c_val,1))*-0.5
    #a = a = np.reshape(np.array([[(e*5/4.5 - 5.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
    #b = np.reshape(a,[100,1])
    b = np.reshape(a,[c_val*c_val,1])
    #c = np.zeros_like(b)
    #c = np.zeros_like(b)
    c = np.zeros_like(b)
    #c = np.zeros_like(b)+8
    #angle
    lcont_sample = np.hstack([b,c])
    # width
    #lcont_sample = np.hstack([c,b])
    
    samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
    if not os.path.exists(sample_directory):
        os.makedirs(sample_directory)
    #Save sample generator images for viewing training progress.
    #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test'+'.png')
    #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test_4'+'.png')
    save_images(np.reshape(samples[0:c_val*c_val],[c_val*c_val,32,32]),[c_val,c_val],sample_directory+'/fig_test_13'+'.png')
""" 